In [10]:
import os
from dotenv import load_dotenv
import streamlit as st
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.vectorstores import Qdrant
from langchain_openai import OpenAIEmbeddings
from langchain.chains import RetrievalQA
from langchain_openai import OpenAI
from qdrant_client import QdrantClient, models
import qdrant_client
from langchain.chat_models import ChatOpenAI
from langchain.chains import ConversationalRetrievalChain, LLMChain
from langchain_core.prompts import PromptTemplate, FewShotPromptTemplate
import logging
logging.basicConfig(level=logging.DEBUG)

In [15]:
# Set the model name for LLM
OPENAI_MODEL = "gpt-3.5-turbo"

# Store API key as a variable
openai_api_key = st.secrets["OPENAI_API_KEY"]

In [11]:
def get_vector_store():
    #Create a client to connect to Qdrant server
    client = qdrant_client.QdrantClient(
        st.secrets["QDRANT_HOST"],
        api_key=st.secrets["QDRANT_API_KEY"]
        )
    
    #initialize embeddings for vector store
    embeddings = OpenAIEmbeddings()
    
    # create vector_store with Qdrant and embeddings
    vector_store = Qdrant(client,
                          collection_name = st.secrets["QDRANT_COLLECTION_NAME"],
                          embeddings=embeddings)
    return vector_store

In [ ]:
qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model=OPENAI_MODEL, temperature=0, api_key=openai_api_key),
    # chain_type="stuff",
    retriever = vector_store.as_retriever(metadata_fields=['metadata'])
    
)

In [24]:
def initialize_crc(vector_store):
    llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0, api_key=openai_api_key)
    retriever=vector_store.as_retriever(metadata_fields=['metadata'])
    crc = ConversationalRetrievalChain.from_llm(llm, retriever)
    st.session_state.crc = crc
    st.success('Source documents loaded!')
    return crc 

In [ ]:
initialize_crc(vector_store)

In [27]:
query = "How does ASC 606 interact with ASC 842?"

response = qa.run(query)

print(response)

NameError: name 'crc' is not defined